In [4]:
!pip install preprocessor
!pip install vaderSentiment


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd
import json
import os
import preprocessor as p
import re
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vincentdandenault/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vincentdandenault/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
csv_path = 'Output/df_cen.csv'
df_input = pd.read_csv(csv_path)

/var/folders/_m/v82wb7j964v3b7b70x4rrvvr0000gn/T/ipykernel_32587/3730228968.py:2: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_input = pd.read_csv(csv_path)


 ## Preprocessing

In [50]:
list_of_countries = ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', \
                   'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', \
                   'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', \
                   'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', \
                   'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', \
                   'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', \
                   'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', \
                   'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', \
                   'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', \
                   'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', \
                   'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", \
                   'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark',
                   'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', \
                   'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', \
                   'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', \
                   'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', \
                   'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', \
                   'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', \
                   'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', \
                   'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', \
                   'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', \
                   'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", \
                   'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", \
                   'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', \
                   'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', \
                   'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', \
                   'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', \
                   'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', \
                   'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', \
                   'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', \
                   'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', \
                   'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', \
                   'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', \
                   'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', \
                   'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia',\
                   'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', \
                   'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', \
                   'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', \
                   'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', \
                   'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', \
                   'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', \
                   'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', \
                   'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', \
                   'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', \
                   'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', \
                   'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', \
                   'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', \
                   'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', \
                   'Zambia', 'Zimbabwe']

def find_country(x):
        for country in list_of_countries:
            if x and country in x:
                return country
        return None
    
def normalize(array):
        return (array - np.min(array)) / (np.max(array) - np.min(array))
    
def preprocess_df(df_input):
    features_worth_keeping = ["text", "truncated", "user",
                    "withheld_in_countries", "entities", "lang",
                    "possibly_sensitive", "extended_tweet"]
    
    df_cen = df_input[features_worth_keeping]
    df_cen['possibly_sensitive'] = df_cen['possibly_sensitive'].fillna(0.0)
    df_raw = df_cen.values
    for line in df_raw:
        line[0] = re.sub(r'http\S+', '', str(line[0])) #remove urls from tweets
        line[0] = re.sub(r'RT @\S+:', '', str(line[0])) #flatten retweets
    
    df_raw = np.delete(df_raw, len(features_worth_keeping)-1, axis=1) #remove "extended_tweet"
    df_raw = np.delete(df_raw, 1, axis=1) #remove "truncated"
    features_worth_keeping.remove("extended_tweet")
    features_worth_keeping.remove("truncated")
    
    for line in df_raw:
        print(type(line[3]))
        line[3] = [x["text"] for x in line[3]["hashtags"]]
    features_worth_keeping[3] = "hashtags"
    
    verified = [line[1]["verified"] for line in dfRaw]
    followers = [line[1]["followers_count"] for line in dfRaw]
    user_id = [line[1]["id"] for line in dfRaw]

    
    location = [find_country(line[1]["location"]) for line in dfRaw]
    
    df_raw = np.c_[df_raw, verified, followers, location, user_id]
    features_worth_keeping += ["verified_account", "followers_count", "location", "user_id"]
    withheld = []
    for line in df_raw:
        if not isinstance(line[2], list):
            line[2] = []
        withheld.append(len(line[2]) != 0)
            
    df_raw = np.c_[df_raw, withheld]
    features_worth_keeping += ["withheld_anywhere"]
    
    sentiment = SentimentIntensityAnalyzer() #we made the assumption that sentiment analysis for this analyzer only works for english
    res = np.array([[x for x in sentiment.polarity_scores(line[0]).values()] if line[4] == "en" 
                    else [0.0, 0.0, 0.0, 0.0] for line in df_raw])

    df_raw = np.c_[df_raw, res]
    features_worth_keeping += ["neg", "neu", "pos", "compound"]
    
    followers_count = np.array([line[1]["followers_count"] for line in df_raw])
    favourites_count = np.array([line[1]["favourites_count"] for line in df_raw])
    statuses_count = np.array([line[1]["statuses_count"] for line in df_raw])
    
    score = (1/3) * (normalize(followers_count) + normalize(favourites_count) + normalize(statuses_count))
    df_raw = np.c_[df_raw, score]
    worthKeeping += ["popularity_score"]
    
    
    return pd.DataFrame(df_raw, columns = worthKeeping)

In [51]:
preprocess_df(df_input)

<class 'str'>


/var/folders/_m/v82wb7j964v3b7b70x4rrvvr0000gn/T/ipykernel_32587/2307944304.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cen['possibly_sensitive'] = df_cen['possibly_sensitive'].fillna(0.0)


TypeError: string indices must be integers

## Feature Encoding

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer

In [ ]:
FIT_CORPUS = True

In [ ]:
features_encoded = ['possibly_sensitive', 'verified_account',\
                    'followers_count', 'user_id', 'neg', 'neu', \
                    'pos', 'compound', 'popularity_score', 'text_embeddings']

In [ ]:
def encode_features(df):
    if FIT_CORPUS:
        corpus = [sentence for sentence in list(df['text'].values)]
        bow_model = CountVectorizer()
        bow_model.fit(corpus)
        bow_texts = bow_model.transform(list(df['text'].values))
        with open('bow_vectors.pickle', 'wb') as pkl:
            pickle.dump(bow_texts, pkl)
        
        tfidf_model = TfidfVectorizer()
        tfidf_model.fit(corpus) 
        tfdif_text = tfidf_model.transform(list(df_english['text'].values)) 
        with open('tfidf_vectors.pickle', 'wb') as pkl:
            pickle.dump(tfidf_vectors, pkl)
        
        model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
        tweets_embeddings = model.encode(sentences=all_tweets, batch_size=32, show_progress_bar=True, convert_to_numpy=True)
        with open('tweets_embeddings.pickle', 'wb') as pkl:
            pickle.dump(tweets_embeddings, pkl)
        
        
    else: 
        with open('bow_vectors.pickle', 'rb') as pkl:
            bow_vectors = pickle.load(pkl)
        with open('tfidf_vectors.pickle', 'rb') as pkl:
            tfdif_text = pickle.load(pkl)
        with open('tweets_embeddings.pickle', 'rb') as pkl:
            tweets_embeddings = pickle.load(pkl)
            
    
    df['bow_vectors'] = bow_vectors
    df['tfidf_vectors'] = tfdif_text
    df['tweets_embeddings'] = tweets_embeddings
    
    country_label = preprocessing.LabelEncoder()
    countries_encoded = country_label.fit_transform(list(df.location.values))
    df['Country_encoded'] = countries_encoded
    
    df = df.astype({"possibly_sensitive": float, "verified_account": float,\
                                'followers_count':int, 'user_id': int, 'neg': float, 'neu': float, 
                               'pos': float, 'compound': float, 'popularity_score': float})
    
    y = df["withheld_anywhere"].astype(int)
    X = df[features_encoded].copy().to_numpy()
    
    return X, y
    
    

In [ ]:
X, y = encode_features(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Training the Models

In [ ]:
seed = 42
models = {'SVM': SVC(random_state=seed), 
          'Random Forest': RandomForestClassifier(random_state=seed), 
          'Gaussian Naive Bayes': GaussianNB()}
scores = {}
for name in models.keys(): 
    y_pred = models[name].fit(X_train, y_train).predict(X_test)
    target_names = ['False', 'True']
    res = classification_report(y_test, y_pred, target_names=target_names) 
    print(name)
    print(res)
    scores[name] = res